# core

> Functions to go a fastq file to a list of genotypes

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *


In [ ]:
#| export
from fastcore.basics import *
from Bio import SeqIO
from Bio import pairwise2
import gzip as gz
import os
from collections import defaultdict, Counter
import numpy as np
import itertools
import click

In [ ]:
#| hide
from DGRec.example_data import get_example_data_dir
data_path=get_example_data_dir()
os.listdir(data_path)

['sacB_ref.fasta', '__pycache__', 'sacB_example.fastq.gz', '__init__.py']

In [ ]:
#| export
def align2mut(align):
    """Converts a sequence alignment result from Bio.pairwise2.Align.globalms into a genotype string.
    Positions are those of the alignment."""
    res=[]
    for i in range(align.end):
        if align.seqA[i]!=align.seqB[i]:
            mut=(align.seqA[i],i,align.seqB[i])
            res.append(mut)
    return res

In [ ]:
seqA = "ATCCCGGCAGC"
seqB = "ATCCACGGTCAGC"
align=pairwise2.align.globalms(seqA,seqB, 2, -1, -1, -.5, one_alignment_only=True)[0]
align2mut(align)

[('-', 4, 'A'), ('-', 8, 'T')]

In [ ]:
#| export
def mut_rix(mutations):
    """Reindexes the positions of the mutations to go from 
    their position in the sequence alignment to their position in the original sequence."""
    ph=0
    res_rix=[]
    for mut in mutations:
        rix=mut[1]+ph
        res_rix.append((mut[0],rix,mut[2]))
        if mut[0]=='-':
            ph-=1
            
    return res_rix

In [ ]:
from Bio.pairwise2 import format_alignment
seqA = "ATCCCGGCAGC"
seqB = "ATCCACGGTCAGC"
align=pairwise2.align.globalms(seqA,seqB, 2, -1, -1, -.5, one_alignment_only=True)[0]
print(format_alignment(*align))

mutations=align2mut(align) 
print("Output of align2mut:")
print(mutations)

print("Output of mut_rix:")
print(mut_rix(mutations))

ATCC-CGG-CAGC
|||| ||| ||||
ATCCACGGTCAGC
  Score=20

Output of align2mut:
[('-', 4, 'A'), ('-', 8, 'T')]
Output of mut_rix:
[('-', 4, 'A'), ('-', 7, 'T')]


In [ ]:
#| export
def get_mutations(seqA,seqB):
    """Aligns two sequences and returns a genotype string.
    The string is a comma separated list of mutations.
    """
    align=pairwise2.align.globalms(seqA,seqB, 2, -1, -1, -.5, one_alignment_only=True)[0]
    mutations=align2mut(align) 
    mutations=mut_rix(mutations)
    return mutations

In [ ]:
seqA = "ATCCCGGCAGCAGGT"
seqB = "ATCCACGGTCAGCACGT"
align=pairwise2.align.globalms(seqA,seqB, 2, -1, -1, -.5, one_alignment_only=True)[0]
print(format_alignment(*align))

get_mutations(seqA,seqB)

ATCC-CGG-CAGCAGGT
|||| ||| |||||.||
ATCCACGGTCAGCACGT
  Score=25



[('-', 4, 'A'), ('-', 7, 'T'), ('G', 12, 'C')]

In [ ]:
#| export

def mut_to_str(mutations: list):
    """Converts list of mutations to a comma separated string"""
    mut_str_list=[''.join(map(str,mut)) for mut in mutations]
    mut_str=','.join(mut_str_list)
    return mut_str

In [ ]:
mutations = get_mutations(seqA,seqB)
print(mutations)
mut_to_str(mutations)

[('-', 4, 'A'), ('-', 7, 'T'), ('G', 12, 'C')]


'-4A,-7T,G12C'

In [ ]:
#| export

def get_UMI_genotype(fastq_path: str, #path to the input fastq file
                     ref_seq: str, #sequence of the reference amplicon
                     umi_size: int = 10, #number of nucleotides at the begining of the read that will be used as the UMI
                     quality_threshold: int = 30, #threshold value used to filter out reads of poor average quality
                     ignore_pos: list = [], #list of positions that are ignored in the genotype
                     ) -> dict:
    
    """Takes as input a fastq_file of single read amplicon sequencing, and a reference amplicon sequence.
       Returns a dictionnary containing as keys UMIs and as values a list of all genotype strings read for that UMI.
    """
    with gz.open(fastq_path,'rt') as handle:
        reads=SeqIO.parse(handle,"fastq")
        n_reads=0
        n_reads_pass_Qfilter=0
        n_reads_aligned=0
        UMI_dict=defaultdict(list,{})
        for r in reads:
            n_reads+=1
            meanScore=np.mean(r.letter_annotations['phred_quality'])

            if meanScore>quality_threshold:
                n_reads_pass_Qfilter+=1
                umi=str(r.seq[:umi_size])
                mutations=get_mutations(ref_seq,r.seq[umi_size:])
                if ignore_pos:
                    mutations = [m for m in mutations if m[1] not in ignore_pos]
                n_mut=len(mutations)
                if n_mut<15: #more than 10 mutation is almost certainly crap
                    n_reads_aligned+=1
                    UMI_dict[umi].append(mut_to_str(mutations))
    
    log='n reads:\t{}\nn_reads pass filter:\t{}\nn_reads aligned:\t{}\n'.format(n_reads,n_reads_pass_Qfilter,n_reads_aligned)
    log+=f"Number of UMIs: {len(UMI_dict)}\n"
    print(log)
    return UMI_dict

In [ ]:
fastq_file="sacB_example.fastq.gz"
fastq_path=os.path.join(data_path,fastq_file)

read_ref_file="sacB_ref.fasta"
ref=next(SeqIO.parse(os.path.join(data_path,read_ref_file),"fasta"))
ref_seq=str(ref.seq)

UMI_dict = get_UMI_genotype(fastq_path, ref_seq, ignore_pos=[0,1,2,138,139,140,141])

for umi in itertools.islice(UMI_dict,20):
    print(umi, UMI_dict[umi])

n reads:	1000
n_reads pass filter:	847
n_reads aligned:	824
Number of UMIs: 814

GCATANCTCA ['A61G,-63T,A76T,A91T']
CGCATNTATA ['']
CCTTGNAGTA ['']
GGCGCNAGAA ['']
TCTCTTGTGA ['']
ATTACAGAAT ['']
CTTTTACTAT ['']
TCAAAGTTTT ['A79T,A91G']
TTAGCTCATA ['']
TCATAATGTA ['']
ATGTGCGGAT ['']
TGTGTTTATA ['']
CCATACATCC ['']
AGGGACGTTT ['A61G,A72G,A76G,A79T']
GTGTAATAGC ['']
ATGTCTTTTA ['']
TATCGGTAGT ['']
GTCGGGGGGG ['']
AAGTGGCACA ['']
AATAGAACCT ['T108A,G127T,G132T']


In [ ]:
#| export

def correct_UMI_genotypes(UMI_dict: dict, #the output of the get_UMI_genotype function
                          reads_thr=2 #only keep UMIs for which we have more than reads_thr reads
                          ) -> dict:
    """Keeps only the genotype with the most reads for each UMI.
    Returns a dictionary with UMIs as keys and a tuple as value: (genotype string, number of reads)
    """
    UMI_gen_dict={}
    for umi in UMI_dict:
        N=len(UMI_dict[umi])
        if N>=reads_thr: #only consider umis for which we have more than reads_thr reads
            gen_counter=Counter(UMI_dict[umi])
            gen=sorted(list(gen_counter.items()),key=lambda x: x[1], reverse=True)[0] #only keep the genotype with the most reads for this umi
            UMI_gen_dict[umi]=gen

    return UMI_gen_dict

In [ ]:
correct_UMI_genotypes(UMI_dict)

{'CTCCGGGGAG': ('', 2),
 'TGCTTGAGTG': ('A79T', 2),
 'AGGGCGGGCT': ('', 2),
 'ATTTCTGTTT': ('', 2),
 'TGGGGGGGCT': ('', 2),
 'GTTAGGGTCT': ('C24G', 1),
 'GATTGGTAGA': ('', 2),
 'GAACTCTAGT': ('', 2),
 'TAACTAATCG': ('A79G,A86G,A91G', 2),
 'GTTGTTCAGT': ('A68C,A72G,A79G,A91G,G94A', 1)}

In [ ]:
#| export

def genotype_UMI_counter(UMI_gen_dict):
    """Takes as input the output of correct_UMI_genotypes() and 
    returns a list of genotypes sorted by the number of UMIs detected corresponding that each genotype."""
    umi_counter=Counter([gen for gen,n in UMI_gen_dict.values()])
    gen_sorted_list=sorted(list(umi_counter.items()),key=lambda x: x[1], reverse=True)
    return gen_sorted_list


In [ ]:
UMI_gen_dict=correct_UMI_genotypes(UMI_dict, reads_thr=0)
gen_list = genotype_UMI_counter(UMI_gen_dict)
for g in gen_list[:20]:
    print(f"{g[1]}\t{g[0]}")

675	
3	C56A
3	A76G
3	A91G
3	A91T
2	C69T
2	T122A
2	A91C
2	A105G
2	C116A
2	T60A
2	T59A
2	A68G
2	T134A
1	A61G,-63T,A76T,A91T
1	A79T,A91G
1	A61G,A72G,A76G,A79T
1	T108A,G127T,G132T
1	A48T,A86G
1	A61T,A68T,A72G,A79C,A91G


In [ ]:
#| export

def get_genotypes(fastq_path: str, #path to the input fastq file
                    ref_seq: str, #sequence of the reference amplicon
                    umi_size: int = 10, #number of nucleotides at the begining of the read that will be used as the UMI
                    quality_threshold: int = 30, #threshold value used to filter out reads of poor average quality
                    ignore_pos: list = [], #list of positions that are ignored in the genotype
                    reads_thr: int = 0, #minimum number of reads required to take a UMI into account. Using a number >2 enables to perform error corrects for UMIs with multiple reads.
                    ):
    """Putting things together in a single wrapper function that takes the fastq as input and returns the list of genotypes."""
    UMI_dict = get_UMI_genotype(fastq_path, ref_seq, umi_size, quality_threshold, ignore_pos)
    UMI_gen_dict=correct_UMI_genotypes(UMI_dict, reads_thr)
    gen_list = genotype_UMI_counter(UMI_gen_dict)
    print("Number of genotypes:", len(gen_list))
    return gen_list
    

In [ ]:
fastq_file="sacB_example.fastq.gz"
fastq_path=os.path.join(data_path,fastq_file)
read_ref_file="sacB_ref.fasta"
ref=next(SeqIO.parse(os.path.join(data_path,read_ref_file),"fasta"))
ref_seq=str(ref.seq)
gen_list = get_genotypes(fastq_path, ref_seq, ignore_pos=[0,1,2,138,139,140,141])
for g in gen_list[:20]:
    print(f"{g[1]}\t{g[0]}")

n reads:	1000
n_reads pass filter:	847
n_reads aligned:	824
Number of UMIs: 814

Number of genotypes: 123
675	
3	C56A
3	A76G
3	A91G
3	A91T
2	C69T
2	T122A
2	A91C
2	A105G
2	C116A
2	T60A
2	T59A
2	A68G
2	T134A
1	A61G,-63T,A76T,A91T
1	A79T,A91G
1	A61G,A72G,A76G,A79T
1	T108A,G127T,G132T
1	A48T,A86G
1	A61T,A68T,A72G,A79C,A91G


In [ ]:
#| export
#Commande line function
@click.command()
@click.argument('fastq', type=click.Path(exists=True))
@click.argument('ref', type=click.Path(exists=True))
@click.option('--umi_size', '-u', default=10, help="Number of nucleotides at the begining of the read that will be used as the UMI")
@click.option('--quality_threshold', '-q', default=10, help="threshold value used to filter out reads of poor average quality")
@click.option('--ignore_pos', '-i', default=[], multiple=True, help="list of positions that are ignored in the genotype, e.g. [0,1,149,150]")
@click.option('--reads_thr', '-r', default=0, help="minimum number of reads required to take a UMI into account. Using a number >2 enables to perform error corrects for UMIs with multiple reads")
@click.option('--output', '-o', default="genotypes.csv", help="output file path")
def DGRec_genotypes(fastq, ref, umi_size, quality_threshold, ignore_pos, reads_thr, output):
    ref=next(SeqIO.parse(ref,"fasta"))
    ref_seq=str(ref.seq)
    gen_list = get_genotypes(fastq, ref_seq, 
                             umi_size=umi_size, 
                             quality_threshold=quality_threshold, 
                             ignore_pos=ignore_pos,
                             reads_thr=reads_thr)
    
    with open(output,"w") as handle:
            for g,n in gen_list:
                handle.write(f"{g}\t{n}\n")

In [ ]:
#| hide
import subprocess

result = subprocess.run(["DGRec_genotypes", 
                         os.path.join(data_path,fastq_file), 
                         os.path.join(data_path,read_ref_file),
                         ])
print(result.stdout)  # Print the standard output of the command
print(result.returncode)  # Get the exit code of the command


# Remove test files

# List all files in the directory
files = os.listdir()

# Iterate over the files
for file in files:
    if file=="genotypes.csv":
        
        try:
            # Delete the file
            os.remove(file)
        except PermissionError:
            print(f"Permission denied to delete file '{file}'.")
        except FileNotFoundError:
            print(f"File '{file}' not found.")

/home/dbikard/anaconda3/envs/DGRec/lib/python3.11/site-packages/Bio/__init__.py:138: BiopythonWarning: You may be importing Biopython from inside the source tree. This is bad practice and might lead to downstream issues. In particular, you might encounter ImportErrors due to missing compiled C extensions. We recommend that you try running your code from outside the source tree. If you are outside the source tree then you have a setup.py file in an unexpected directory: /home/dbikard/anaconda3/envs/DGRec/lib/python3.11/site-packages
  warnings.warn(
/home/dbikard/anaconda3/envs/DGRec/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


n reads:	1000
n_reads pass filter:	955
n_reads aligned:	912
Number of UMIs: 902

Number of genotypes: 185
None
0


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()